In [9]:
from helper.sql_engine import get_engine
from sqlalchemy import text
from dash import Dash, html, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc


In [10]:
engine = get_engine()

# Use the database
with engine.connect() as connection:
    connection.execute(text("USE alzheimer;"))
# Load data from the database
patients = pd.read_sql('SELECT * FROM patient', engine)
lifestyle = pd.read_sql('SELECT * FROM lifestyle', engine)
clinical_measurements = pd.read_sql('SELECT * FROM clinical_measurements', engine)
medical_history = pd.read_sql('SELECT * FROM medical_history', engine)
cognitive_assessments = pd.read_sql('SELECT * FROM cognitive_assessments', engine)
symptoms = pd.read_sql('SELECT * FROM symptoms', engine)
alzheimers_data = pd.read_sql('SELECT * FROM alzheimers_data', engine)


In [11]:
# Merge datasets into a single DataFrame
all_data = patients.merge(
    medical_history, on="PatientID"
).merge(
    lifestyle, on="PatientID"
).merge(
    clinical_measurements, on="PatientID"
).merge(
    cognitive_assessments, on="PatientID"
).merge(
    symptoms, on="PatientID"
).merge(
    alzheimers_data, on="PatientID"
)

# Lifestyle and diagnosis analysis
life_style_diagnosis = lifestyle.merge(alzheimers_data)

In [12]:
# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc

# Incorporate data
df = round(life_style_diagnosis)

# Initialize the app - incorporate a Dash Bootstrap theme
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__, external_stylesheets=external_stylesheets)

# App layout
app.layout = dbc.Container([
    dbc.Row([
        html.Div('My First App with Data, Graph, and Controls', className="text-primary text-center fs-3")
    ]),

    dbc.Row([
        dbc.RadioItems(options=[{"label": x, "value": x} for x in ['SleepQuality', 'BMI', 'Smoking']],
                       value='BMI',
                       inline=True,
                       id='radio-buttons-final')
    ]),

    dbc.Row([
        dbc.Col([
            dash_table.DataTable(data=df[['SleepQuality', 'BMI', 'Smoking']].to_dict('records'), page_size=12, style_table={'overflowX': 'auto'})
        ], width=6),

        dbc.Col([
            dcc.Graph(figure={}, id='my-first-graph-final')
        ], width=6),
    ]),

], fluid=True)

# Add controls to build the interaction
@callback(
    Output(component_id='my-first-graph-final', component_property='figure'),
    Input(component_id='radio-buttons-final', component_property='value')
)
def update_graph(col_chosen):
    fig = px.bar(df, x='Diagnosis', y=col_chosen, barmode="group")
    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)


In [13]:
# Incorporate data
df = round(all_data,2)
df["Diagnosis2"] = df["Diagnosis"].map({0: "No Alzheimer", 1: "Alzheimer"})

# Define consistent color mapping
color_map = {
    "No Alzheimer": "#17becf",
    "Alzheimer": "#ff7f0e"
}

# Initialize the app with a Bootstrap theme
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__, external_stylesheets=external_stylesheets)

# App layout
app.layout = dbc.Container([
    dbc.Row(html.H2('Alzheimer Diagnosis Analysis', className="text-primary text-center mb-4")),

    # Filters
    dbc.Row([
        dbc.Col([
            html.H4('Gender & Ethnicity Filter', className="text-primary", style={"margin-left": "10px"}),
            dcc.Dropdown(
                id='gender-ethnicity-filter',
                options=[
                    {'label': 'Male', 'value': '0'},
                    {'label': 'Female', 'value': '1'},
                    {'label': 'Caucasian', 'value': '2'},
                    {'label': 'African American', 'value': '3'},
                    {'label': 'Asian', 'value': '4'},
                    {'label': 'Other', 'value': '5'}
                ],
                value=['0', '1'],
                multi=True,
                clearable=True,
                className='mb-4'
            )
        ], width=4),
        dbc.Col([
            html.H4('Age Filter', className="text-primary", style={"margin-left": "20px"}),
            dcc.RangeSlider(
                id='age-filter',
                min=60,
                max=90,
                step=1,
                marks={i: str(i) for i in range(60, 91, 5)},
                value=[60, 90],
                className='mb-4'
            )
        ], width=4),
                dbc.Col([
            html.H4('Diagnosis Filter', className="text-primary", style={"margin-left": "20px"}),
            dcc.Checklist(
                id='diagnosis-filter',
                    options=[
                {'label': html.Span(' Alzheimer ', 
                                    style={'backgroundColor': color_map["Alzheimer"], 
                                        'padding': '5px 10px', 
                                        'borderRadius': '5px', 
                                        'color': 'white',
                                        'fontWeight': 'bold',
                                        'margin': '5px'}), 
                'value': 1},
                {'label': html.Span(' No Alzheimer ', 
                                    style={'backgroundColor': color_map["No Alzheimer"], 
                                        'padding': '5px 10px', 
                                        'borderRadius': '5px', 
                                        'color': 'white',
                                        'fontWeight': 'bold',
                                        'margin': '5px'}), 
                'value': 0}
            ],
                value=[0,1],  # Default selected options
                inline=True,  # Display options inline
                className='mb-4',
                style={'display': 'flex', 'justify-content': 'center', 'gap': '30px'}
            )
        ], width=4),
    ]),

    # Graphs
    dbc.Row([
        dbc.Col(dcc.Graph(figure={}, id='scatter-plot'), width=8),
        dbc.Col(dcc.Graph(figure={}, id='bar-plot'), width=4),
    ]),

], fluid=True)

# Callback function
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('bar-plot', 'figure')],
    [Input('gender-ethnicity-filter', 'value'),
     Input("age-filter", "value"),
     Input("diagnosis-filter", "value")]
)
def update_graph(gender_ethnicity_values, age_values, diagnosis_values):
    # Extract gender and ethnicity values
    gender_values = [0, 1]
    ethnicity_values = [0, 1, 2, 3]
    min_age, max_age = age_values

    # Adjust filters based on user input
    if any(val in ['0', '1'] for val in gender_ethnicity_values):
        gender_values = [int(val) for val in gender_ethnicity_values if val in ['0', '1']]
    if any(val in ['2', '3', '4', '5'] for val in gender_ethnicity_values):
        ethnicity_values = [int(val) - 2 for val in gender_ethnicity_values if val in ['2', '3', '4', '5']]
    if any(val in ['0', '1'] for val in diagnosis_values):
        diagnosis_values = [int(val) for val in diagnosis_values if val in ['0', '1']]
        

    # Filter data
    filtered_df = df[df['Gender'].isin(gender_values) & df['Ethnicity'].isin(ethnicity_values) & 
                     (df['Age'] >= min_age) & (df['Age'] <= max_age) & df['Diagnosis'].isin(diagnosis_values)].sort_values(by="Diagnosis")

    # Scatter Plot: Physical Activity vs BMI
    scatter_fig = px.scatter(
        filtered_df, 
        x='PhysicalActivity', 
        y='BMI', 
        color=filtered_df['Diagnosis2'].astype(str),
        hover_data={'Age': True, 'Ethnicity': True, 'Diagnosis2': True},
        color_discrete_map=color_map  # Apply consistent color mapping
    )

    scatter_fig.update_traces(
        marker=dict(size=5, opacity=0.8, line=dict(width=1, color='DarkSlateGrey'))
    )

    scatter_fig.update_layout(
        title="Impact of Physical Activity and BMI on Diagnosis",
        xaxis_title="Physical Activity",
        yaxis_title="BMI",
        legend_title="Diagnosis",
        xaxis=dict(gridcolor='lightgray'),
        yaxis=dict(gridcolor='lightgray'),
        showlegend=False,
        margin=dict(t=50, b=50, l=50, r=50),
        plot_bgcolor="white"
    )

    # Bar Plot: Diagnosis Distribution
    bar_fig = px.histogram(
        filtered_df, 
        x=filtered_df['Diagnosis2'].astype(str), 
        color=filtered_df['Diagnosis2'].astype(str),
        color_discrete_map=color_map  # Apply the same color mapping
    )

    bar_fig.update_layout(
        title="Diagnosis Distribution",
        xaxis_title="Diagnosis",
        yaxis_title="Count",
        showlegend=False,
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=True, gridcolor='lightgray'),
        margin=dict(t=50, b=50, l=50, r=50),
        plot_bgcolor="white"
    )

    return scatter_fig, bar_fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)
